In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn.model_selection import train_test_split
from torch import nn, optim

from torchvision import models, datasets
from torchvision import transforms
from torch import nn, optim
from torch.utils.data.dataloader import DataLoader
import copy
from torch.optim import lr_scheduler
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# applying transforms to the data
#Statistics Based on ImageNet Data for Normalisation
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

image_transforms = {"train":transforms.Compose([
                                transforms.Resize((128,128)), #Resizes all images into same dimension
                                transforms.RandomRotation(10), # Rotates the images upto Max of 10 Degrees
                                transforms.RandomHorizontalFlip(p=0.4), #Performs Horizantal Flip over images 
                                transforms.ToTensor(), # Coverts into Tensors
                                transforms.Normalize(mean = mean_nums, std=std_nums)]), # Normalizes
                    "valid": transforms.Compose([
                                transforms.Resize((128,128)),
                                transforms.CenterCrop(150), #Performs Crop at Center and resizes it to 150x150
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean_nums, std = std_nums)
                    ])}

In [ ]:
# Load data
# Set train, valid, and test directory
def create_data(bs,n):
    
    

    train_directory = 'chest_xray/train'
    valid_directory = 'chest_xray/test'






    data = {
        'train': datasets.ImageFolder(root=train_directory,
                                    transform=image_transforms['train']),
        'valid': datasets.ImageFolder(root=valid_directory,
                                    transform=image_transforms['valid']),
    }



    # size of data, to be used for calculating Averge Loss and Accuracy
    train_data_size = len(data['train'])
    valid_data_size = len(data['valid'])

    # Convert labels to 0s and 1s
    class_to_idx = data['train'].class_to_idx
    
    data['train'].targets = [1 if data['train'].targets[i] == class_to_idx['PNEUMONIA'] else 0 for i in range(len(data['train'].targets))]
    data['valid'].targets = [1 if data['valid'].targets[i] == class_to_idx['PNEUMONIA'] else 0 for i in range(len(data['valid'].targets))]

    

    # Create iterators for the Data loaded using DataLoader module
    train_data = DataLoader(data['train'], batch_size=bs, shuffle=True)
    valid_data = DataLoader(data['valid'], batch_size=1, shuffle=True)

    
    hold=[]
    trainloader=[]
    testloader=[]

    k=(train_data_size/bs)/(n)
    
    for i, (inputs, labels) in enumerate(train_data):
        hold.append([inputs,labels])
        
        if i % int(k) == 0 and i != 0:
            
            trainloader.append(hold)
            hold=[]

    for i, (inputs, labels) in enumerate(valid_data):
        testloader.append([inputs,labels])

            

    return trainloader, testloader, valid_data_size

In [ ]:
class ModelA(nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.resnet50 = models.densenet121(pretrained=True)
        self.act1 = nn.Sigmoid() 
        for param in self.resnet50.parameters():
            param.requires_grad = True
        self.fc1 = nn.Linear(1000, 512)
    def forward(self, x):
        x = self.resnet50(x)
        output = self.act1(self.fc1(x))
        return output

class ModelB(nn.Module):
    def __init__(self):
        super(ModelB, self).__init__()


        
        # Create an instance of ModelA
        self.act1 = nn.Sigmoid()  
        # Additional layers for ModelB (classification or other operations)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
   
        
    def forward(self, x):
        # Get the output of ModelA

        x = self.act1(self.fc2(x))
        x = self.act1(self.fc3(x))



        
        return x

class ModelC(nn.Module):
    def __init__(self, modelA, modelB):
        super(ModelC, self).__init__()
        self.model_A = modelA
        self.model_B = modelB
        self.classification_layer = nn.Linear(128, 1)
        
    def forward(self, x):
        # Forward pass through ModelA and then ModelB
        x = self.model_A(x)
        x = self.model_B(x)
        x = self.classification_layer(x)
        return x




In [ ]:
import torch.optim as optim


def train(model, epochs, train_data, lr):
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model = model.to(device)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        
        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            
            # Ensure labels are in the right format (adjust if needed)
            # For example, if outputs are logits, convert labels to one-hot encoding
            
            # Compute loss
            loss = loss_func(outputs.squeeze(), labels.float())  # Assuming outputs are logits
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_data)




def test(model, val_data, val_size):

    loss_func = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification

    valid_loss = 0
    valid_acc = 0
    model = model.to(device)
    model.eval()  # Set the model to evaluation mode
    
    with torch.no_grad():
        for j, (inputs, labels) in enumerate(val_data):
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)  # Ensure labels are in the correct shape

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = loss_func(outputs, labels)  # Calculate BCE loss
            valid_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            predictions = (torch.sigmoid(outputs) >= 0.5).float()  # Convert logits to binary predictions (0 or 1)
            correct_counts = (predictions == labels).sum().item()
            valid_acc += correct_counts

    return valid_loss / val_size, valid_acc / val_size


      
           
            



            


In [ ]:
def num_model(n):
    model_list=[]
    for i in range(n):
        modelA_instance = ModelA()
        modelB_instance = ModelB()  

        start_model = ModelC(modelA_instance, modelB_instance)
     
        model_list.append(start_model)
    return model_list


In [ ]:
def train_global_standard(n,k,n_epochs,batch_size):

    trainloaders,val_data,val_size=create_data(batch_size,n)
    


    mod_list=num_model(n)
    acc=[]
    epo=[]
    curr=0
    
    best_value=0
    for i in range(k):

        for z in range(n):
            train(mod_list[z],n_epochs,trainloaders[z],0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)


        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test(mod_list[i],val_data,val_size)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])

        print(res_acc)
        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])  
  
        if avg_value > best_value:
            best_value = avg_value

        
        parameter_lists=[]
        for model in mod_list:
            hold=model.model_B
            parameter_lists.append(hold.parameters())





        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

            

        glob_hold = ModelB()

            
        updated_params = []

            
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

        
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)



        
        for model_instance in mod_list:
            model_instance.model_B = glob_hold





    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with Five Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()
    return best_value

In [ ]:
train_global_standard(5,50,5,32)

In [ ]:
def train_global_Imp(n,k,n_epochs,batch_size):

    trainloaders,val_data,val_size=create_data(batch_size,n)
    


    
    

    mod_list=num_model(n)
    for i in range(n):
        train(mod_list[i],100,trainloaders[i],0.001)
    
        


    

    


    best_avg_acc=0
    acc=[]
    epo=[]
    curr=0
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,trainloaders[i],0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test(mod_list[i],val_data,val_size)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])    

        if avg_value > best_avg_acc:
            best_avg_acc = avg_value
        parameter_lists=[]
        k=0
        for model in mod_list:
            if res_acc[k]>avg_value:
                print("fedder")
                hold=model.model_B
                parameter_lists.append(hold.parameters())
            k=k+1





        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

            

        glob_hold = ModelB()

            
        updated_params = []

            
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

        
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)



        
        for model_instance in mod_list:
            model_instance.model_B = glob_hold

        # Freeze all layers in glob_hold
        if curr % 5 == 0:
            print("fixer")
            
            for param in glob_hold.parameters():
                param.requires_grad = False
        

            for model_instance in mod_list:
                model_instance.model_B = glob_hold
                

      
            k=0
            for i in range(n):
                if res_acc[k]<avg_value:
                    print("fed_fix")
                    train(mod_list[i],100,trainloaders[i],0.003)
                k=k+1

            for model_instance in mod_list:
                for param in model_instance.parameters():
                    param.requires_grad = True
                

            print(test(mod_list[i],val_data,val_size))
        else:
            for model_instance in mod_list:
                model_instance.model_B = glob_hold




        

         
            







        



    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with Five Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()
    return best_avg_acc


In [ ]:
train_global_Imp(5,50,5,32)

In [ ]:
def train_Fed(n,Itterations,n_epochs,batch_size):

    mod_list=num_model(n)
    
    trainloaders,val_data,val_size=create_data(batch_size,n)
    print(len(trainloaders))
    modelA_instance = ModelA()
    modelB_instance = ModelB()  

    glob_model = ModelC(modelA_instance, modelB_instance)
    acc=[]
    epo=[]
    curr=0
    best_value=0
    for i in range(Itterations):

        for i in range(n):
            train(mod_list[i],n_epochs,trainloaders[i],0.001)

        parameter_lists = [model.parameters() for model in mod_list]

        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        for glob_param, param in zip(glob_model.parameters(), avg_params):
            glob_param.data.copy_(param)

        mod_list=[]

        for i in range(n):
            mod_list.append(glob_model)


        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(1):
            hold=test(mod_list[i],val_data,val_size)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])

        

        avg_value=sum(res_acc)
        avg_loss=sum(res_loss)

        if avg_value>best_value:
            best_value=avg_value


        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])
        




        



    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.title("Accuracy of Image Categorization")
    plt.legend()

    plt.show()
    print(best_value)
    return best_value

In [ ]:
train_Fed(1,20,5,32)